In [1]:
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import BooleanType, IntegerType
from datetime import *
from settings import obtener_timestamp, obtener_dia_semana

In [2]:
""" Configuramos Spark """
conf = SparkConf()
conf.setAppName("ProcesamientoDatos")
conf.setMaster("local[*]")

In [3]:
spark = SparkSession.builder.config(conf=conf).getOrCreate()

Leemos los datos procesados del archivo guardado

In [4]:
data = spark.read.format('parquet').load('./../datos/processed/full.parquet/')

In [5]:
data.printSchema()

root
 |-- medallon: string (nullable = true)
 |-- licencia: string (nullable = true)
 |-- hora_subida: timestamp (nullable = true)
 |-- hora_bajada: timestamp (nullable = true)
 |-- duracion_viaje_seg: integer (nullable = true)
 |-- longitud_subida: decimal(18,14) (nullable = true)
 |-- latitud_subida: decimal(18,14) (nullable = true)
 |-- longitud_bajada: decimal(18,14) (nullable = true)
 |-- latitud_bajada: decimal(18,14) (nullable = true)
 |-- tipo_pago: string (nullable = true)
 |-- tarifa: decimal(10,2) (nullable = true)
 |-- propina: decimal(10,2) (nullable = true)
 |-- cantidad_total: decimal(10,2) (nullable = true)
 |-- cuad_latitud_subida: long (nullable = true)
 |-- cuad_longitud_subida: long (nullable = true)
 |-- cuad_latitud_bajada: long (nullable = true)
 |-- cuad_longitud_bajada: long (nullable = true)
 |-- dia_semana: integer (nullable = true)



In [6]:
tiempo_fin = obtener_timestamp("2013-01-01", "01:30")
tiempo_inicio = tiempo_fin - timedelta(minutes=30)

### Consulta 1: Rutas frequentes
En esta primera búsqueda lo que vamos a obtener es las 10 rutas más frecuentes durante los 30 minutos anteriores. Estas rutas contarán únicamente si el viaje ha sido completado, es decir, si el usuario se ha bajado del taxi. La salida de la consulta será la siguiente:

    hora_subida, hora_bajada, celda_subida_1, celda_bajada_1, ..., celda_subida_10, celda_bajada_10, tiempo_ejecucion

In [7]:
mejor = data.filter(data.hora_subida <= tiempo_fin) \
    .filter(data.hora_subida >= tiempo_inicio) \
    .filter(data.hora_bajada <= tiempo_fin) \
    .filter(data.hora_bajada >= tiempo_inicio) \
    .groupBy("cuad_longitud_subida", "cuad_latitud_subida", "cuad_longitud_bajada", "cuad_latitud_bajada") \
    .count().orderBy(desc("count"))

In [8]:
mejor = mejor.take(10)

In [9]:
mejor

[Row(cuad_longitud_subida=159, cuad_latitud_subida=161, cuad_longitud_bajada=158, cuad_latitud_bajada=163, count=6),
 Row(cuad_longitud_subida=157, cuad_latitud_subida=156, cuad_longitud_bajada=158, cuad_latitud_bajada=154, count=6),
 Row(cuad_longitud_subida=157, cuad_latitud_subida=162, cuad_longitud_bajada=159, cuad_latitud_bajada=161, count=5),
 Row(cuad_longitud_subida=156, cuad_latitud_subida=166, cuad_longitud_bajada=157, cuad_latitud_bajada=164, count=5),
 Row(cuad_longitud_subida=158, cuad_latitud_subida=162, cuad_longitud_bajada=157, cuad_latitud_bajada=164, count=5),
 Row(cuad_longitud_subida=159, cuad_latitud_subida=161, cuad_longitud_bajada=161, cuad_latitud_bajada=157, count=5),
 Row(cuad_longitud_subida=161, cuad_latitud_subida=158, cuad_longitud_bajada=162, cuad_latitud_bajada=156, count=5),
 Row(cuad_longitud_subida=162, cuad_latitud_subida=156, cuad_longitud_bajada=159, cuad_latitud_bajada=161, count=5),
 Row(cuad_longitud_subida=158, cuad_latitud_subida=163, cuad_lon

### Consulta 1B: Rutas frequentes
En esta primera búsqueda lo que vamos a obtener es las 10 rutas más frecuentes en los dias de una semana durante los 30 minutos anteriores a una fecha dada. Estas rutas contarán únicamente si el viaje ha sido completado, es decir, si el usuario se ha bajado del taxi. La salida de la consulta será la siguiente:

    hora_subida, hora_bajada, celda_subida_1, celda_bajada_1, ..., celda_subida_10, celda_bajada_10, tiempo_ejecucion

In [10]:
dia_elegido = obtener_dia_semana("Lunes")

Debido a diversas limitaciones de spark utilizamos variables globales para hacer las limitaciones de tiempo

In [3]:
hora_fin = datetime.strptime("00:30:00", "%H:%M:%S")
hora_inicio = (hora_fin - timedelta(minutes=30))


def comparar_hora(hora):
    """
        Metodo que filtra las horas de los registros para que concuerden
        con las horas de busqueda deseada
        :param hora: Timestamp completo
        :return: True si las horas del timestamp estan entre las deseadas
        False si lo contrario
    """
    if hora.time() <= hora_fin.time() and hora.time() >= hora_inicio.time():
        return True
    return False


def relevancia(fecha):
    """
        Metodo que da mas relevancia a los viajes mas cercanos a la
        fecha de busqueda deseada.
        Si la diferencia es menor a un mes de la fecha
        dada los registros tienen más relevancia
        :param fecha: Timestamp completo
        :return: 2 si el viaje esta cerca de la fecha deseada, 1 si no
    """
    diferencia = fecha - tiempo_fin
    if diferencia < timedelta(days=30) and diferencia > timedelta(days=-30):
        return 2
    else:
        return 1

comprobar_hora = udf(comparar_hora, BooleanType())
calcular_relevancia = udf(relevancia, IntegerType())

In [57]:
filtered =  data.filter(data.dia_semana == dia_elegido) \
    .filter(comprobar_hora(data.hora_subida)) \
    .filter(comprobar_hora(data.hora_bajada)) \
    .withColumn('relevancia', calcular_relevancia(data.hora_subida))

In [58]:
frequent = filtered.groupBy("cuad_longitud_subida", "cuad_latitud_subida", \
                            "cuad_longitud_bajada", "cuad_latitud_bajada") \
    .sum("relevancia") \
    .select(col("cuad_longitud_subida"), col("cuad_latitud_subida"), \
            col("cuad_longitud_bajada"), col("cuad_latitud_bajada"), \
            col("sum(relevancia)").alias("frecuencia")) \
    .orderBy("frecuencia", ascending=False)

In [60]:
filtered.show()


+--------------------+--------------------+--------------------+--------------------+------------------+------------------+-----------------+------------------+-----------------+---------+------+-------+--------------+-------------------+--------------------+-------------------+--------------------+----------+----------+
|            medallon|            licencia|         hora_subida|         hora_bajada|duracion_viaje_seg|   longitud_subida|   latitud_subida|   longitud_bajada|   latitud_bajada|tipo_pago|tarifa|propina|cantidad_total|cuad_latitud_subida|cuad_longitud_subida|cuad_latitud_bajada|cuad_longitud_bajada|dia_semana|relevancia|
+--------------------+--------------------+--------------------+--------------------+------------------+------------------+-----------------+------------------+-----------------+---------+------+-------+--------------+-------------------+--------------------+-------------------+--------------------+----------+----------+
|254B4453B29C73B8E...|C8908B05D

In [55]:
mes = "06"
hora = "15:00"
HORA_FIN = datetime.strptime(mes + " " + hora, "%m %H:%M")


In [56]:
print(HORA_FIN)

1900-06-01 15:00:00
